### asyncio 예제

In [1]:
import asyncio

In [9]:
async def hello():
    # 네이티브 코루틴 생성
    
    print('Hello, World!')

In [10]:
loop = asyncio.get_event_loop() #이벤트 루프 get
loop.run_until_complete(hello()) #hello가 끝날때까지 기다림 
loop.close() #루프 close

RuntimeError: This event loop is already running

Hello, World!


**run_until_complete**
- 네이티브 코루틴을 호출하면 코루틴 객체 생성
- 네이티브 코루틴이 이벤트 루프에서 실행되도록 예약
- 해당 네이티브 코루틴이 끝날 때까지 wait

***************

### await 예제
- await 뒤에 코루틴 객체, 퓨처객체, 태스크 객체를 지정하면 해당 객체가 끝날 때까지 기다린 뒤 결과를 반환
- await는 네이티브 코루틴 안에서만 사용

In [11]:
import asyncio

async def add(a, b):
    print('add: {0} + {1}'.format(a, b))
    await asyncio.sleep(0.5)
    #없어도 되는데 비동기로 작동하는거 확인하려고 씀
    #asyncio.sleep 도 네이티브 코루틴이라서 await를 사용
    
    return a + b

In [12]:
async def print_add(a, b):
    result = await add(a, b) 
    #코루틴 안에서 다른 코루틴을 실행할 때는 await사용
    print('print_add : {0} + {1} = {2}'.format(a, b, result))

In [13]:
loop_1 = asyncio.get_event_loop()
loop_1.run_until_complete(print_add(1, 2))
loop_1.close(0)

RuntimeError: This event loop is already running

add: 1 + 2
print_add : 1 + 2 = 3


**********

### asyncio 사용하지 않고 urllib.requet의 urlopen으로 웹페이지 크롤링, 웹페이지 길이 출력

In [14]:
from time import time
from urllib.request import Request, urlopen

urls = ['https://www.google.co.kr/search?q=' + i
        for i in ['apple', 'pear', 'grape', 'pineapple', 'orange', 'strawberry']]

begin = time()
result = []
for url in urls:
    request = Request(url, headers={'User-Agent' : 'Mozilla/5.0'}) #UA 없으면 403에러
    response = urlopen(request)
    page = response.read()

    result.append(len(page))

In [15]:
print(result)
end = time()
print('실행시간 : {0:.3f}초'.format(end-begin))
#웹페이지를 하나를 완전히 가져온 뒤에 다음 웹페이지를 가져옴

[98236, 90326, 90695, 92661, 101304, 91210]
실행시간 : 6.025초


### asyncio로 웹페이지 호출 및 크기 출력

**run_in_executor**
- urlopen이나 response.read 같은 함수는 결과가 나올 때까지 코드 실행이 중단: 블로킹I/O(Blockin I/O)
- 네이티브 코루틴 안에서 블로킹 I/O 함수실행 : 이벤트 루프의 run_in_executor함수 사용 -> 다른 스레드에서 병렬 실행

**eunsure_future**
- 태스크 객체 생성, 리스트로 만들어줌

**gather**
- 태스크 리스트를 gather 함수에 넣어줌
- 모든 코루틴 객체가 끝날 때까지 기다린 위 결과를 리스트로 반환
- 리스트가 아닌 위치 인수로 객체를 받음
- asyncio.gater(*futures)와 같이 리스트를 언패킹해서 넣어줌

In [1]:
from time import time
from urllib.request import Request, urlopen
import asyncio

urls = ['https://www.google.co.kr/search?q=' + i
        for i in ['apple', 'pear', 'grape', 'pineapple', 'orange', 'strawberry']]

async def fetch(url):
    request = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    response = await loop.run_in_executor(None, urlopen, request)
    page = await loop.run_in_executor(None, response.read)
    return len(page)
#이벤트루프.run_in_executor(None, 함수, 인수1, 인수2, 인수3)


async def main():
    futures = [asyncio.ensure_future(fetch(url)) for url in urls]
    result = await asyncio.gather(*futures)
    print(result)
#     return result

In [2]:
begin = time()
loop_1 = asyncio.get_event_loop()
loop_1.run_until_complete(main())
loop_1.close()
end = time()

print('실행시간 : {0:.3f}초'.format(end-begin))

RuntimeError: This event loop is already running

*********

### async with (그냥 Session?)

**\__aenter__ 와 \__aexit__**
- \__aenter__ 에 들어간 뒤 return으로 값 반환
- async with에서 as에 지정한 변수에 \__aenter__ 반환값 들어감
- \__aexit__ 는 async with as를 완전이 벗어나면 호출 (메서드 자체가 없으면 에러 발생)

In [19]:
import asyncio

class AsyncAdd:
    def __init__(self, a, b):
        self.a = a
        self.b = b
        
    async def __aenter__(self):
        await asyncio.sleep(1.0)
        return self.a + self.b
    
    async def __aexit__(self, exc_type, exc_value, traceback):
        pass
    
async def main():
    async with AsyncAdd(1, 2) as result:
        print(result)

In [20]:
loop = asyncio.get_event_loop()
loop.run_until_complete(main())
loop.close()

RuntimeError: This event loop is already running

3


*********

### async for

**\__aiter__와 \__anext__**
- StopAsyncIteration : 반복으 끝낼 때 발생시키는 예외
- for 앞에 async

In [34]:
import asyncio

class AsyncCounter:
    def __init__(self, stop):
        self.current = 0
        self.stop = stop
        
    def __aiter__(self):
        return self
    
    async def __anext__(self):
        if self.current < self.stop:
            await asyncio.sleep(1.0)
            r = self.current
            self.current += 1
            return r
        else:
            raise StopAsyncIteration
            
async def main():
    async for i in AsyncCounter(3):
        print(i, end = ' ')

In [35]:
loop = asyncio.get_event_loop()
loop.run_until_complete(main())
loop.close()

RuntimeError: This event loop is already running

0 1 2 

*******************

### yield

In [4]:
def test1(i):
    print('start test1 coroutine')
    while True:
        yield i
        i += 1

a = test1(5)
next(a)    # start test1 coroutine 출력 후 5출력, yield i 부분에서 멈춰있다.
next(a)    # 멈춰진 yield i 부분 다음줄의 5 += 1(i=6)을 수행한후 다음 6을 출력하고 yield i에서 멈춘다.
next(a)    # 7을 출력하고 yield i에서 멈춘다.
next(a)
next(a)

start test1 coroutine


9

In [11]:
next(a)

16

In [12]:
def test2(i):
    print('start test2 coroutine')
    while True:
        value = yield i 
        i += value

b = test2(5)
next(b)    # start test1 coroutine 출력 후 5출력, yield i 부분에서 멈춰있다.
b.send(3)    # yield를 통해 3을 전달하여 value가 3이 된다. 이후 i += value 줄을 거쳐 i=8이되고 한바퀴 돌아 8을출력, yield에서 멈춘다. 
b.send(5)    # 5를 더해 i는 8이되고 8을 출력한다.

start test2 coroutine


13